In [57]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import json

#### First Step: Filtering the data
This step is needed only once

In [58]:
#data from https://covid.saude.gov.br/
#df = pd.read_csv('HIST_PAINEL_COVIDBR_2022_Parte1_27jun2022.csv', sep=';', on_bad_lines='skip')

In [59]:
#df_states = df[~df['estado'].isna()]
#df_states = df[(~df['estado'].isna()) & (df['codmun'].isna())] #< removes data from cities and leaves only states
#df_brazil = df[df['regiao'] == 'Brasil']

In [60]:
#df_states.to_csv('df_states.csv')
#df_brazil.to_csv('df_brazil.csv')

#### Second Step: Setting Parameters and Visualizing Data 

In [61]:
#Central positions
CENTER_LAT, CENTER_LON = -14.272572694355336,-51.25567404158474

In [62]:
df_states = pd.read_csv('df_states.csv')
df_brazil = pd.read_csv('df_brazil.csv')
df_data = df_states[df_states['estado']=='SP'] 

In [63]:
df_states.columns

Index(['Unnamed: 0', 'regiao', 'estado', 'municipio', 'coduf', 'codmun',
       'codRegiaoSaude', 'nomeRegiaoSaude', 'data', 'semanaEpi',
       'populacaoTCU2019', 'casosAcumulado', 'casosNovos', 'obitosAcumulado',
       'obitosNovos', 'Recuperadosnovos', 'emAcompanhamentoNovos',
       'interior/metropolitana'],
      dtype='object')

In [64]:
select_columns = {'casosAcumulado':'Casos Acumulado',
'casosNovos':'Novos Casos',
'obitosAcumulado':'Obitos Totais',
'obitosNovos':'Obitos por dia'}

In [65]:
brazil_states = json.load(open('geojson/brazil_geo.json', 'r'))
type(brazil_states)
brazil_states.keys()
type(brazil_states['features'])
type(brazil_states['features'][0])

dict

In [66]:
brazil_states['features'][0].keys()

dict_keys(['type', 'id', 'properties', 'geometry'])

In [67]:
#brazil_states['features'][0]['geometry']

In [68]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

In [69]:
df_states.head()

,Unnamed: 0,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,178,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-01,52,1777225.0,284660.0,34,6737,3,NaN,NaN,NaN
1,179,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-02,1,1777225.0,284692.0,32,6739,2,NaN,NaN,NaN
2,180,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-03,1,1777225.0,284783.0,91,6742,3,NaN,NaN,NaN
3,181,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-04,1,1777225.0,285037.0,254,6745,3,NaN,NaN,NaN
4,182,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-05,1,1777225.0,285269.0,232,6748,3,NaN,NaN,NaN


In [70]:
df_states_ = df_states[df_states['data'] == '2022-01-01']

In [71]:
#Map 
fig = px.choropleth_mapbox(df_states, locations='estado', color='casosNovos', geojson='brazil_states',
                           center= {'lat': -16.95, 'lon': -47.78},
                           zoom=4,
                           color_continuous_scale='Redor', opacity=0.4, 
                           hover_data={'casosAcumulado':True, 'casosNovos':True, 'obitosNovos':True, 'estado':True})

fig.update_layout(
    paper_bgcolor='#242424',
    autosize=True,
    margin=dict(l=10,r=10,b=10,t=10),
    showlegend=False,
    mapbox_style='carto-darkmatter'
)

#Data 
fig2 = go.Figure(layout={'template': 'plotly_dark'})
fig2.add_trace(go.Scatter(x=df_data['data'], y=df_data['casosAcumulado']))
fig2.update_layout(
    paper_bgcolor='#242424',
    plot_bgcolor='#242424',
    autosize=True,
    margin=None
)

#### Third step: Bulding the App, Functions and Server

In [72]:
app.layout = dbc.Container(
    dbc.Row([
        
       dbc.Col([
            html.Div([
                html.Img(id='logo', src=app.get_asset_url('logo.png'), height=50),
                html.H5('Covid-19 Evolution'),
                dbc.Button('BRASIL', color='primary', id='location-button', size='lg') 
            ], style={}),
           html.P('Informe a data na qual deseja obter informações:', style={'margin-top': '40px'}),
           html.Div(id='div-test',children=[
               dcc.DatePickerSingle(
                   id='date-picker',
                   min_date_allowed=df_brazil['data'].min(),
                   max_date_allowed=df_brazil['data'].max(),
                   initial_visible_month=df_brazil['data'].min(),
                   date=df_brazil['data'].max(),
                   display_format='MMM D, YYYY',
                   style={'border' : '0px solid black'}
               )
           ]),
           
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span('Casos recuperados'),
                            html.H4(style={'color':'#adfc92'}, id='casos-recuperados-text'),
                            html.Span('Em acompanhamento'),
                            html.H6(id='em-acompanhamento-text'),
                        ])
                    ], color='light', outline=True, style={'margin-top':'10px',
                                     'box-shadow': '0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0,0,0,0.19)',
                                     'color':'#FFFFFF'})
                ], md=4),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span('Casos confirmados totais'),
                            html.H4(style={'color':'#389fd6'}, id='casos-confirmados-text'),
                            html.Span('Novos casos na data'),
                            html.H6(id='novos-casos-text'),
                        ])
                    ], color='light', outline=True, style={'margin-top':'10px',
                                     'box-shadow': '0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0,0,0,0.19)',
                                     'color':'#FFFFFF'})
                ], md=4),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span('Óbitos confirmadoss'),
                            html.H4(style={'color':'#df2935'}, id='obitos-text'),
                            html.Span('Óbitos na data'),
                            html.H6(id='obitos-na-data-text'),
                        ])
                    ], color='light', outline=True, style={'margin-top':'10px',
                                     'box-shadow': '0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0,0,0,0.19)',
                                     'color':'#FFFFFF'})
                ], md=4),
           ]),
           
           html.Div([
               html.P('Selecione o tipo de dado que deseja visualizar:', style={'margin-top': '25px'}),
               dcc.Dropdown(id='location-dropdown',
                            options=[{'label': j,'value': i} for i, j in select_columns.items()],
                            value='casosNovos',
                            style={'margin-top':'10px'}
                           ),
           ]),
           
           dcc.Graph(id='line-graph', figure=fig2)
        ], md=5, style={'padding': '25px', 'background-color': '#242424'}),
        
        dbc.Col([
            dcc.Loading(id='loading-1', type='default',
            children=[
                dcc.Graph(id='choropleth-map', figure=fig, style={'height': '100vh', 'margin-right' : '10px'})])
        ], md=7)
    ], class_name='g-0')
, fluid=True)

#Interactivity cards
@app.callback(
   [
       Output('casos-recuperados-text','children'),
       Output('em-acompanhamento-text','children'),
       Output('casos-confirmados-text','children'),
       Output('novos-casos-text','children'),
       Output('obitos-text','children'),
       Output('obitos-na-data-text','children'),
   ],
   [Input('date-picker','date'), Input('location-button','children')]
   )

def display_status(date, location):
    if location =='BRASIL':
        df_data_on_date= df_brazil[df_brazil['data']== date]
    
    else:
        df_data_on_date = df_states[(df_states['estado'] == location) & (df_states['data'] == date)]
        
    df_data_on_date['Recuperadosnovos']
    recuperados_novos= '-' if df_data_on_date['Recuperadosnovos'].isna().values[0] else f"{int(df_data_on_date['Recuperadosnovos'].values[0]):,}".replace(',','.')
    acompanhamentos_novos= '-' if df_data_on_date['emAcompanhamentoNovos'].isna().values[0] else f"{int(df_data_on_date['emAcompanhamentoNovos'].values[0]):,}".replace(',','.')
    casos_acumulados= '-' if df_data_on_date['casosAcumulado'].isna().values[0] else f"{int(df_data_on_date['casosAcumulado'].values[0]):,}".replace(',','.')
    casos_novos= '-' if df_data_on_date['casosNovos'].isna().values[0] else f"{int(df_data_on_date['casosNovos'].values[0]):,}".replace(',','.')
    obitos_acumulado = '-' if df_data_on_date['obitosAcumulado'].isna().values[0] else f"{int(df_data_on_date['obitosAcumulado'].values[0]):,}".replace(',','.')
    obitos_novos= '-' if df_data_on_date['obitosNovos'].isna().values[0] else f"{int(df_data_on_date['obitosNovos'].values[0]):,}".replace(',','.')                                          
    
    return (recuperados_novos,
            acompanhamentos_novos,
            casos_acumulados,
            casos_novos,
            obitos_acumulado,
            obitos_novos)

#Update the graph according to type

@app.callback(Output("line-graph","figure"),
       [
           Input("location-dropdown","value"), Input("location-button","children"),
       ]
       )
def plot_line_graph(plot_type, location):
    if location =="BRASIL":
        df_data_on_location = df_brazil.copy()
    else:
        df_data_on_location = df_states[df_states["estado"] == location]
        
    bar_plots = ["casosNovos", "obitosNovos"]
    
    fig2 = go.Figure(layout={"template":"plotly_dark"})
    
    if plot_type in bar_plots:
        fig2.add_trace(go.Bar(x=df_data_on_location['data'], y=df_data_on_location[plot_type]))
    else:
        fig2.add_trace(go.Scatter(x=df_data_on_location['data'], y=df_data_on_location[plot_type]))
    
    fig2.update_layout(
        paper_bgcolor="#242424",
        plot_bgcolor="#242424",
        autosize=True,
        margin=dict(l=10,r=10,b=10,t=10),
    )
    return fig2

# Update map according to the date

@app.callback(
   Output("choropleth-map","figure"),
    [Input("date-picker","date")]
)
def update_map(date):
    df_data_on_states = df_states[df_states["data"] == date]
    
    fig = px.choropleth_mapbox(df_data_on_states, locations="estado", geojson=brazil_states,
       center={"lat":CENTER_LAT,"lon":CENTER_LON},
       zoom=4, color="casosAcumulado", color_continuous_scale="Redor", opacity=0.55,
       hover_data={"casosAcumulado":True,"casosNovos":True,"obitosNovos":True,"estado":False}
       )
    
    fig.update_layout(paper_bgcolor="#242424", mapbox_style="carto-darkmatter", autosize=True,
                  margin = go.layout.Margin(l=0, r=8, t=8, b=0), showlegend=False)
    return fig


#Reset to Brazil the data

@app.callback(
    Output('location-button', 'children'),
    [Input('choropleth-map', 'clickData'), Input('location-button', 'n_clicks')]
)

def update_location(click_data, n_clicks):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if click_data is not None and changed_id != 'location-button.n_clicks':
        state = click_data['points'][0]['location']
        return '{}'.format(state)
    else:
        return 'BRASIL'

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
